# List of Exposures with Hologram at USDF


- work with Weakly_2023_01
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/19
- last update : 2023/07/21



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-6.0.0/bin/python3


In [2]:
print(sys.path)

['/sdf/data/rubin/user/dagoret/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2023_27/atmospec/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/ctrl_mpexec/g88183d3282+88d805ed8c/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/spectractor/g7bf34a7a6e/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/obs_lsst/gfd7e96728c+6f6aa72e7b/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/pipe_tasks/gfb267207ab+bb26f3254a/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/shapelet/gd4112516d0+5ce2cc5508/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/meas_modelfit/g807e3ba568+b7d80ac9cf/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/scarlet_extensions/g9d18589735+d445b4966c/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/proxmin/g33b4157f25/l

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_27/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-6.0.0/eups 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

Weekly 2023_27
   gdf42428520+4e9f6d16c8 	current w_2023_27 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [6]:
from IPython.display import display, HTML

In [7]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [8]:
import lsst.daf.butler as dafButler

In [9]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [10]:
collection='LATISS/raw/all'

## List of Exposures

In [11]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','azimuth','zenith','science_program','jd','mjd'])

In [12]:
print(butler.registry.dimensions["exposure"].RecordClass.fields)

exposure: 
  instrument: str
  id: int
  physical_filter: str
  obs_id: str
  exposure_time: float
  dark_time: float
  observation_type: str
  observation_reason: str
  day_obs: int
  seq_num: int
  seq_start: int
  seq_end: int
  group_name: str
  group_id: int
  target_name: str
  science_program: str
  tracking_ra: float
  tracking_dec: float
  sky_angle: float
  azimuth: float
  zenith_angle: float
  has_simulated: bool
  timespan: lsst.daf.butler.Timespan


In [13]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,pd.to_datetime(info.timespan.begin.to_string()),pd.to_datetime(info.timespan.end.to_string()) ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info.timespan.begin.jd,info.timespan.begin.mjd ]
    
    if count < 5:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        mjd = Time(info.timespan.begin.to_string()).mjd
        jd = Time(info.timespan.begin.to_string()).jd
        print(mjd,jd)
    

----------------------------------------------------- 0 ---------------------------------------------------------
exposure:
  instrument: 'LATISS'
  id: 2022083100004
  physical_filter: 'unknown~unknown'
  obs_id: 'AT_O_20220831_000004'
  exposure_time: 0.0
  dark_time: 0.0160482
  observation_type: 'bias'
  observation_reason: 'bias'
  day_obs: 20220831
  seq_num: 4
  seq_start: 4
  seq_end: 4
  group_name: '2022083100004'
  group_id: 2022083100004
  target_name: 'UNKNOWN'
  science_program: 'unknown'
  tracking_ra: None
  tracking_dec: None
  sky_angle: None
  azimuth: None
  zenith_angle: None
  has_simulated: False
  timespan: Timespan(begin=astropy.time.Time('2022-08-31 22:58:13.633984', scale='tai', format='iso'), end=astropy.time.Time('2022-08-31 22:58:13.650000', scale='tai', format='iso'))
	 id:                   2022083100004
	 day_obs:              20220831
	 seq_num:              4
	 type-of-observation:  bias
	 target:               UNKNOWN
59822.957102245186 2459823.45710

In [14]:
info.toDict()

{'instrument': 'LATISS',
 'id': 2023072000411,
 'physical_filter': 'SDSSg_65mm~empty',
 'obs_id': 'AT_O_20230720_000411',
 'exposure_time': 30.0,
 'dark_time': 30.2357,
 'observation_type': 'science',
 'observation_reason': 'object',
 'day_obs': 20230720,
 'seq_num': 411,
 'seq_start': 411,
 'seq_end': 411,
 'group_name': '2023-07-21T05:42:43.948',
 'group_id': 2698477639480000,
 'target_name': 'Photo1800-1_018',
 'science_program': 'AUXTEL_PHOTO_IMAGING',
 'tracking_ra': 270.1606190868831,
 'tracking_dec': -24.985955840249968,
 'sky_angle': 359.99767545605386,
 'azimuth': 267.03058313505875,
 'zenith_angle': 38.41494881028694,
 'has_simulated': False,
 'timespan': Timespan(begin=astropy.time.Time('2023-07-21 05:43:39.125015', scale='tai', format='iso'), end=astropy.time.Time('2023-07-21 05:44:09.361000', scale='tai', format='iso'))}

In [15]:
info

exposure.RecordClass(instrument='LATISS', id=2023072000411, physical_filter='SDSSg_65mm~empty', obs_id='AT_O_20230720_000411', exposure_time=30.0, dark_time=30.2357, observation_type='science', observation_reason='object', day_obs=20230720, seq_num=411, seq_start=411, seq_end=411, group_name='2023-07-21T05:42:43.948', group_id=2698477639480000, target_name='Photo1800-1_018', science_program='AUXTEL_PHOTO_IMAGING', tracking_ra=270.1606190868831, tracking_dec=-24.985955840249968, sky_angle=359.99767545605386, azimuth=267.03058313505875, zenith_angle=38.41494881028694, has_simulated=False, timespan=Timespan(begin=astropy.time.Time('2023-07-21 05:43:39.125015', scale='tai', format='iso'), end=astropy.time.Time('2023-07-21 05:44:09.361000', scale='tai', format='iso')))

In [16]:
info.azimuth

267.03058313505875

In [17]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51890,2023072000407,AT_O_20230720_000407,20230720,407,2023-07-21 05:40:36.937992,2023-07-21 05:41:07.171,science,Photo1800-1_017,SDSSr_65mm~empty,37.769487,30.0,270.144342,-24.985933,359.997734,267.367196,37.769487,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.236539
51891,2023072000408,AT_O_20230720_000408,20230720,408,2023-07-21 05:41:21.267987,2023-07-21 05:41:51.532,science,Photo1800-1_017,SDSSg_65mm~empty,37.928194,30.0,270.144203,-24.985929,359.997829,267.284076,37.928194,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.237052
51892,2023072000409,AT_O_20230720_000409,20230720,409,2023-07-21 05:42:05.638014,2023-07-21 05:42:35.871,science,Photo1800-1_017,SDSSi_65mm~empty,38.090330,30.0,270.144245,-24.985954,0.001695,267.199353,38.090330,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.237565
51893,2023072000410,AT_O_20230720_000410,20230720,410,2023-07-21 05:42:54.730003,2023-07-21 05:43:24.977,science,Photo1800-1_018,SDSSi_65mm~empty,38.252772,30.0,270.160636,-24.985984,359.997601,267.114762,38.252772,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.238133


In [18]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

In [19]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51890,2023072000407,AT_O_20230720_000407,20230720,407,2023-07-21 05:40:36.937992,2023-07-21 05:41:07.171,science,Photo1800-1_017,SDSSr_65mm~empty,37.769487,30.0,270.144342,-24.985933,359.997734,267.367196,37.769487,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.236539
51891,2023072000408,AT_O_20230720_000408,20230720,408,2023-07-21 05:41:21.267987,2023-07-21 05:41:51.532,science,Photo1800-1_017,SDSSg_65mm~empty,37.928194,30.0,270.144203,-24.985929,359.997829,267.284076,37.928194,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.237052
51892,2023072000409,AT_O_20230720_000409,20230720,409,2023-07-21 05:42:05.638014,2023-07-21 05:42:35.871,science,Photo1800-1_017,SDSSi_65mm~empty,38.090330,30.0,270.144245,-24.985954,0.001695,267.199353,38.090330,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.237565
51893,2023072000410,AT_O_20230720_000410,20230720,410,2023-07-21 05:42:54.730003,2023-07-21 05:43:24.977,science,Photo1800-1_018,SDSSi_65mm~empty,38.252772,30.0,270.160636,-24.985984,359.997601,267.114762,38.252772,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.238133


## Selection of science

In [20]:
df_science = df_exposure[df_exposure.type == 'science']

In [21]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_4782/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [22]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
51835,2023072000352,AT_O_20230720_000352,20230720,352,2023-07-21 04:57:51.978016,2023-07-21 04:58:22.224,science,Photo1800-1_031,SDSSr_65mm~empty,28.445749,30.0,270.242100,-24.956352,359.998911,272.911641,28.445749,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.206852
51836,2023072000353,AT_O_20230720_000353,20230720,353,2023-07-21 04:58:42.031014,2023-07-21 04:59:12.270,science,Photo1800-1_032,SDSSr_65mm~empty,28.608159,30.0,270.258545,-24.956391,359.998714,272.801891,28.608159,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.207431
51837,2023072000354,AT_O_20230720_000354,20230720,354,2023-07-21 04:59:26.316994,2023-07-21 04:59:56.548,science,Photo1800-1_032,SDSSg_65mm~empty,28.770397,30.0,270.258499,-24.956374,359.998767,272.692973,28.770397,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.207943
51838,2023072000355,AT_O_20230720_000355,20230720,355,2023-07-21 05:00:10.638016,2023-07-21 05:00:40.875,science,Photo1800-1_032,SDSSi_65mm~empty,28.928947,30.0,270.258543,-24.956414,359.998684,272.587047,28.928947,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.208456
51839,2023072000356,AT_O_20230720_000356,20230720,356,2023-07-21 05:00:57.880005,2023-07-21 05:01:28.113,science,Photo1800-1_019,SDSSi_65mm~empty,29.163268,30.0,270.176910,-24.985940,359.998684,272.367732,29.163268,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.209003
51840,2023072000357,AT_O_20230720_000357,20230720,357,2023-07-21 05:01:42.234019,2023-07-21 05:02:12.476,science,Photo1800-1_019,SDSSg_65mm~empty,29.321981,30.0,270.176826,-24.985937,359.998850,272.263631,29.321981,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.209517
51841,2023072000358,AT_O_20230720_000358,20230720,358,2023-07-21 05:02:26.539995,2023-07-21 05:02:56.792,science,Photo1800-1_019,SDSSr_65mm~empty,29.480546,30.0,270.176908,-24.985972,359.998655,272.160137,29.480546,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.210029
51842,2023072000359,AT_O_20230720_000359,20230720,359,2023-07-21 05:03:14.044988,2023-07-21 05:03:44.292,science,Photo1800-1_030,SDSSr_65mm~empty,29.620503,30.0,270.225869,-24.956380,359.998622,272.132438,29.620503,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.210579
51843,2023072000360,AT_O_20230720_000360,20230720,360,2023-07-21 05:03:58.385001,2023-07-21 05:04:28.626,science,Photo1800-1_030,SDSSg_65mm~empty,29.775509,30.0,270.225932,-24.956376,359.998627,272.032106,29.775509,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.211092
51844,2023072000361,AT_O_20230720_000361,20230720,361,2023-07-21 05:04:42.714996,2023-07-21 05:05:12.957,science,Photo1800-1_030,SDSSi_65mm~empty,29.937934,30.0,270.225785,-24.956315,359.998866,271.927664,29.937934,AUXTEL_PHOTO_IMAGING,2.460147e+06,60146.211605


In [23]:
df_science.day_obs.unique()

array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013, 20221025, 20221026, 20221027,
       20221108, 20221109, 20221110, 20221122, 20221123, 20221124,
       20221207, 20221208, 20221209, 20221210, 20221212, 20230117,
       20230118, 20230119, 20230131, 20230201, 20230202, 20230214,
       20230215, 20230216, 20230228, 20230301, 20230302, 20230314,
       20230315, 20230316, 20230509, 20230510, 20230511, 20230523,
       20230524, 20230525, 20230704, 20230705, 20230706, 20230718,
       20230720])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [24]:
#date_sel = 20220630
#date_sel = 20220630
#date_sel = 20221208
#date_sel = 20221209

# data in january
#date_sel= 20230118
#date_sel= 20230131

# data in February
#date_sel = 20230201
#date_sel = 20230202
#date_sel = 20230704

#date_sel = 20230510
#date_sel = 20230509
#date_sel = 20230511

#date_sel = 20230705
#date_sel = 20230706

#july 2023
#date_sel = 20230718
date_sel = 20230720

## Query Dimension records on Exposures

In [25]:
where_clause = f"instrument=\'LATISS\' AND exposure.day_obs={date_sel} AND exposure.science_program=\'spec\'"

In [26]:
results_exposures = registry.queryDimensionRecords('exposure',datasets="raw",collections = 'LATISS/raw/all',where=where_clause)

## Select date

In [27]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [28]:
len(df_science_selected)

215

# List of filters

In [29]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['SDSSr_65mm~empty', 'SDSSg_65mm~empty', 'SDSSi_65mm~empty',
       'empty~holo4_003', 'empty~blue300lpmm_qn1'], dtype=object)

In [30]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003'], dtype='<U15')

## Selection of filter

- But now we have a loop on filters

In [31]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [32]:
pd.set_option('display.max_rows', None)

In [33]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20230720_filt_empty-holo4_003.list


/tmp/ipykernel_4782/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)


In [34]:
N = len(all_def_science_selected_final)

In [35]:
if N>=1:
    n0=len(all_def_science_selected_final[0])
    display(all_def_science_selected_final[0])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
51717,2023072000234,AT_O_20230720_000234,20230720,234,2023-07-21 02:05:58.761013,2023-07-21 02:06:28.999,science,HD146233,empty~holo4_003,25.897148,30.0,243.866033,-8.399073,239.914285,324.180775,25.897148,spec,2.460147e+06,60146.087486
51718,2023072000235,AT_O_20230720_000235,20230720,235,2023-07-21 02:06:41.809019,2023-07-21 02:07:12.054,science,HD146233,empty~holo4_003,25.991896,30.0,243.865998,-8.398970,239.914326,323.819198,25.991896,spec,2.460147e+06,60146.087984
51721,2023072000238,AT_O_20230720_000238,20230720,238,2023-07-21 02:08:28.797012,2023-07-21 02:08:59.045,science,HD146233,empty~holo4_003,26.217545,30.0,243.868678,-8.402749,234.914230,322.962186,26.217545,spec,2.460147e+06,60146.089222
51722,2023072000239,AT_O_20230720_000239,20230720,239,2023-07-21 02:09:11.854996,2023-07-21 02:09:42.096,science,HD146233,empty~holo4_003,26.313812,30.0,243.868535,-8.402546,234.914441,322.613320,26.313812,spec,2.460147e+06,60146.089721
51731,2023072000248,AT_O_20230720_000248,20230720,248,2023-07-21 02:14:23.521984,2023-07-21 02:14:53.770,science,HD146233,empty~holo4_003,27.012890,30.0,243.867858,-8.401359,236.122991,320.219621,27.012890,spec,2.460147e+06,60146.093328
51732,2023072000249,AT_O_20230720_000249,20230720,249,2023-07-21 02:15:06.540017,2023-07-21 02:15:36.781,science,HD146233,empty~holo4_003,27.114808,30.0,243.867873,-8.401375,236.123007,319.888796,27.114808,spec,2.460147e+06,60146.093826
51736,2023072000253,AT_O_20230720_000253,20230720,253,2023-07-21 02:35:45.995012,2023-07-21 02:36:16.233,science,HD185975,empty~holo4_003,57.842451,30.0,307.769004,-87.433803,42.623966,177.835843,57.842451,spec,2.460147e+06,60146.108171
51737,2023072000254,AT_O_20230720_000254,20230720,254,2023-07-21 02:36:29.007010,2023-07-21 02:36:59.244,science,HD185975,empty~holo4_003,57.836603,30.0,307.768586,-87.433818,42.624264,177.842823,57.836603,spec,2.460147e+06,60146.108669
51739,2023072000256,AT_O_20230720_000256,20230720,256,2023-07-21 02:37:47.189004,2023-07-21 02:38:17.426,science,HD185975,empty~holo4_003,57.826045,30.0,307.769508,-87.433785,42.623093,177.855431,57.826045,spec,2.460147e+06,60146.109574
51740,2023072000257,AT_O_20230720_000257,20230720,257,2023-07-21 02:38:30.397017,2023-07-21 02:39:00.643,science,HD185975,empty~holo4_003,57.820249,30.0,307.769941,-87.433772,42.622620,177.862419,57.820249,spec,2.460147e+06,60146.110074


In [36]:
if N>=2:
    n1=len(all_def_science_selected_final[1])
    display(all_def_science_selected_final[1])

In [37]:
if N>=3:
    n2=len(all_def_science_selected_final[2])
    display(all_def_science_selected_final[2])

In [38]:
if N>=4:
    n3=len(all_def_science_selected_final[3])
    display(all_def_science_selected_final[3])

In [39]:
if N>=5:
    n4=len(all_def_science_selected_final[4])
    display(all_def_science_selected_final[4])

In [40]:
df_forsave = pd.concat(all_def_science_selected_final)

In [41]:
filename_out_csv = "holosummary_"+ str(date_sel) + ".csv"
filename_out_excel = "holosummary_"+ str(date_sel) + ".xlsx"

In [42]:
filename_out_csv

'holosummary_20230720.csv'

In [43]:
df_forsave.to_csv(filename_out_csv) 

In [44]:
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [45]:
df_forsave.to_excel(filename_out_excel) 